In [6]:
from dotenv import load_dotenv

load_dotenv()

True

# LCEL (대화내용 기억하기): 메모리 추가

In [7]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# 대화형 프롬프트를 생성. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 사용자 입력을 포함
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [8]:
# 대화 버퍼 메모리를 생성, 메시지 반환 기능을 활성화
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [9]:
memory.load_memory_variables({})

{'chat_history': []}

In [10]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")
)

In [11]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [14]:
runnable.invoke({"input": "hi"})
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful chatbot"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")]
)

In [15]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [16]:
chain = runnable | prompt | model

In [17]:
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 종진입니다."})
print(response.content)

만나서 반갑습니다, 종진님! 어떻게 도와드릴까요?


In [18]:
memory.load_memory_variables({})

{'chat_history': []}

In [19]:
# 입력된 데이터와 응답 내용을 메모리에 저장
memory.save_context({"human": "만나서 반갑습니다. 제 이름은 종진입니다."}, {"ai": response.content})

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 종진입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='만나서 반갑습니다, 종진님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

In [20]:
# 이름을 기억하고 있는지 추가 질의
response = chain.invoke({"input": "제 이름이 무엇이었는지 기어하세요?"})
response.content

'종진님이십니다! 다른 질문이나 도움이 필요하시면 말씀해 주세요.'

# 커스텀 ConversationChain 구현 예시

In [21]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [25]:
class MyConversationChain(Runnable):
    def __init__(self, llm, prompt, memory, input_key="input"):
        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key
        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer


In [26]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 대화형 프롬프트를 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

conversation_chain = MyConversationChain(llm, prompt, memory)

In [27]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제이름은 종진입니다.")

'안녕하세요, 종진님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [28]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'종진님이라고 하셨습니다. 맞나요?'

In [29]:
conversation_chain.invoke("앞으로는 영어롬나 답변해주세요. 알겠어요?")

'Got it! I will respond in English from now on. How can I assist you today?'

In [30]:
conversation_chain.invoke(" 제 이름을 다시 한 번 말해주세요")

'Your name is Jongjin.'

In [32]:
conversation_chain.memory.load_memory_variables({})['chat_history']

[HumanMessage(content='안녕하세요? 만나서 반갑습니다. 제이름은 종진입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 종진님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='종진님이라고 하셨습니다. 맞나요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='앞으로는 영어롬나 답변해주세요. 알겠어요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Got it! I will respond in English from now on. How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' 제 이름을 다시 한 번 말해주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Jongjin.', additional_kwargs={}, response_metadata={})]